In [6]:
# ZERO_SHOT_REACT_DESCRIPTION 을 이용한 Open AI function 에제
from langchain_openai import ChatOpenAI
from langchain.tools import StructuredTool, Tool
from langchain.agents import initialize_agent, AgentType

from example.test111 import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1)


# ZERO_SHOT_REACT_DESCRIPTION 은 1개의 파라메터 만 받을 수 있다
# STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION + StructuredTool 을 이용할 때에는
# 연산이 중간에 끊기 거나 잘못된 결과가 나오는 경우가 있었는데 ZERO_SHOT_REACT_DESCRIPTION + Tool 
# 을 이용했을 때에는 그런경우가 나오지 않았음, 수식 처리에서는 이게 효과적인건가?
def plus(param_csv):
    a, b = param_csv.split(",")
    return float(a) + float(b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    tools=[
        Tool.from_function(
            func=plus,
            name="Sum Calculator",
            description="Use this to perform sums of two numbers. Use this tool by sending a pair of number separated by a comma.\n"
                        "Example:1,2",
        ),
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

# llm 자체는 연산을 수행할 수 없다 
agent.invoke(prompt)



> Entering new AgentExecutor chain...
This is a sum of multiple numbers, so I should use the Sum Calculator tool to find the total cost.
Action: Sum Calculator
Action Input: 355.39,924.87
Observation: 1280.26
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 1280.26,721.2
Observation: 2001.46
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 2001.46,1940.29
Observation: 3941.75
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 3941.75,573.63
Observation: 4515.38
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 4515.38,65.72
Observation: 4581.1
Thought:I need to continue adding the remaining numbers to the total.
Action: Sum Calculator
Action Input: 4581.1,35.00
Observation: 4616.1
Thought:I need to continue adding the remaining numbers to the total.
Action: S

{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': '$5273.38'}

In [3]:
355.39 + 924.87 + 721.2 + 1940.29 + 573.63 + 65.72 + 35.00 + 552.00 + 76.16 + 29.12

5273.38

In [2]:
# pydantic 을 이용 하여 OPEN AI Function 구조를 정의 하고 BaseTool 을 상속 받아 새로운 Tool 을 생성 하는 예제
from typing import Type, Any
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel, Field

llm = ChatOpenAI(temperature=0.1)


def plus(a, b):
    return a + b


# function 의 구조를 설정 예전 quiz gpt 에서는 json 형식 으로 일일히 지정해 줬지만 pydantic 을 이용 하여 정의 하는 것도 가능  
class CalculatorToolArgsSchema(BaseModel):
    a: float = Field(description="The first number")
    b: float = Field(description="The second number")


# function call 을 사용 하면 token 의 개수를 획기적 으로 줄일 수 있다
class CalculatorTool(BaseTool):
    name = "SumsCalculator"
    description = """
        Use this to perform sums of two numbers.
        The first and second arguments should be numbers.
        Only receives two arguments.
    """
    args_schema: Type[CalculatorToolArgsSchema] = CalculatorToolArgsSchema

    def _run(self, a, b) -> float:
        return plus(a, b)


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=[
        CalculatorTool()
    ],
)

prompt = "Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12"

# llm 자체는 연산을 수행할 수 없다 
agent.invoke(prompt)

C:\Dev\py_workspace\fullstack-gpt\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...

Invoking: `SumsCalculator` with `{'a': 355.39, 'b': 924.87}`


1280.26
Invoking: `SumsCalculator` with `{'a': 1280.26, 'b': 721.2}`


2001.46
Invoking: `SumsCalculator` with `{'a': 2001.46, 'b': 1940.29}`


3941.75
Invoking: `SumsCalculator` with `{'a': 3941.75, 'b': 573.63}`


4515.38
Invoking: `SumsCalculator` with `{'a': 4515.38, 'b': 65.72}`


4581.1
Invoking: `SumsCalculator` with `{'a': 4581.1, 'b': 35.0}`


4616.1
Invoking: `SumsCalculator` with `{'a': 4616.1, 'b': 552.0}`


5168.1
Invoking: `SumsCalculator` with `{'a': 5168.1, 'b': 76.16}`


5244.26
Invoking: `SumsCalculator` with `{'a': 5244.26, 'b': 29.12}`


5273.38The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.

> Finished chain.


{'input': 'Cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12',
 'output': 'The total cost of $355.39 + $924.87 + $721.2 + $1940.29 + $573.63 + $65.72 + $35.00 + $552.00 + $76.16 + $29.12 is $5273.38.'}

In [9]:
import os
import requests
from typing import Type, Any
from langchain_core.tools import BaseTool
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from pydantic import BaseModel, Field
from langchain.utilities import DuckDuckGoSearchAPIWrapper

llm = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo-1106")

alpha_vantage_api_key = os.getenv("ALPHAVANTAGE_API_KEY")


class StockMarketSymbolSearchToolArgsSchema(BaseModel):
    query: str = Field(description="The query you will search for")


class StockMarketSymbolSearchTool(BaseTool):
    name = "StockMarketSymbolSearchTool"
    description = """
    Use this tool to find the stock market symbol for a company.
    It takes a query as an argument.
    """
    args_schema: Type[StockMarketSymbolSearchToolArgsSchema] = StockMarketSymbolSearchToolArgsSchema

    def _run(self, query) -> Any:
        ddg = DuckDuckGoSearchAPIWrapper()
        return ddg.run(query)


class CompanyOverViewToolArgsSchema(BaseModel):
    symbol: str = Field(description="Stock Symbol of the company. Example: APPL, TSLA")


# 회사의 요약 정보
class CompanyOverViewTool(BaseTool):
    name = "CompanyOverViewTool"
    description = """
    Use this to get an overview of the financials of the company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyOverViewToolArgsSchema] = CompanyOverViewToolArgsSchema

    def _run(self, symbol) -> Any:
        result = requests.get(
            f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}')
        return result.json()


class CompanyIncomeStatementToolArgsSchema(BaseModel):
    symbol: str = Field(description="Stock Symbol of the company. Example: APPL, TSLA")


# 회사의 손익 계산서
class CompanyIncomeStatementTool(BaseTool):
    name = "CompanyIncomeStatementTool"
    description = """
    Use this to get the income statement of a company.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyIncomeStatementToolArgsSchema] = CompanyIncomeStatementToolArgsSchema

    def _run(self, symbol) -> Any:
        result = requests.get(
            f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}')
        return result.json()['annualReports']


class CompanyTimeSeriesWeeklyToolArgsSchema(BaseModel):
    symbol: str = Field(description="Stock Symbol of the company. Example: APPL, TSLA")


# 주간 주가
class CompanyTimeSeriesWeeklyTool(BaseTool):
    name = "CompanyTimeSeriesWeeklyTool"
    description = """
    Use this to get the weekly performance of a company stock.
    You should enter a stock symbol.
    """
    args_schema: Type[CompanyTimeSeriesWeeklyToolArgsSchema] = CompanyTimeSeriesWeeklyToolArgsSchema

    def _run(self, symbol) -> Any:
        result = requests.get(
            f'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY&symbol={symbol}&apikey={alpha_vantage_api_key}')
        org_json = result.json()['Weekly Time Series']
        return {k: v for k, v in org_json.items() if k >= '2024-03-22'}


agent = initialize_agent(
    llm=llm,
    verbose=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=[
        StockMarketSymbolSearchTool(),
        CompanyOverViewTool(),
        CompanyIncomeStatementTool(),
        CompanyTimeSeriesWeeklyTool(),
    ],
)

prompt = ("Give me information on DataDog's stock, considering its financials and income statements and stock performance "
          "help me analyze if it's a potential good investment")

# llm 자체는 연산을 수행할 수 없다 
agent.invoke(prompt)



> Entering new AgentExecutor chain...

Invoking: `StockMarketSymbolSearchTool` with `{'query': 'DataDog'}`


NEW YORK — Datadog, Inc. (NASDAQ: DDOG), the monitoring and security platform for cloud applications, today announced the general availability of IT Event Management to its suite of AIOps capabilities.With Event Management, Datadog intelligently consolidates, correlates and enriches all alert events and important signals from Datadog and existing third-party observability tools into one ... NEW YORK — Datadog, Inc. (NASDAQ: DDOG), the monitoring and security platform for cloud applications, today announced it has been named a Leader in the 2023 Gartner Magic Quadrant for Application Performance Monitoring and Observability.This is the third year in a row Gartner has positioned Datadog as a Leader in its Magic Quadrant. The Gartner Magic Quadrant for Application Performance ... NEW YORK, Nov. 7, 2023 /PRNewswire/ -- Datadog, Inc. (NASDAQ:DDOG), the monitoring and security platf

{'input': "Give me information on DataDog's stock, considering its financials and income statements and stock performance help me analyze if it's a potential good investment",
 'output': "Here's the information on Datadog's stock:\n\n1. **Company Overview:**\n   - Datadog, Inc. (NASDAQ: DDOG) provides an analytics and monitoring platform for developers, IT operations teams, and business users in the cloud. The company is headquartered in New York, New York.\n   - Market Capitalization: $37,795,041,000\n   - Revenue (TTM): $2,257,898,000\n   - Gross Profit (TTM): $1,328,623,000\n   - Earnings Per Share (EPS): $0.34\n   - Price-to-Earnings Ratio (P/E): 332.32\n   - Forward P/E: 83.33\n   - Analyst Target Price: $147.34\n   - 52-Week High: $138.61\n   - 52-Week Low: $77.81\n\n2. **Income Statement:**\n   - The company has shown consistent growth in revenue and gross profit over the past few years.\n   - In 2023, the total revenue was $2,122,251,000, with a gross profit of $1,718,451,000.\